<a href="https://colab.research.google.com/github/jrmirmi/ML-phenotype-pred/blob/main/Dataset_logistic_regression_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pickle
import pandas as pd
import numpy as np
import random
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/GitHub')
from labelling import label_ADNI
import gc
from sklearn.model_selection import KFold

In [31]:
with open('/content/drive/MyDrive/PROJECT/NCBI_graph_PET.pickle', 'rb') as handle:
    lista_dizionari = pickle.load(handle)


# Percorso del file GZIP
file_path = '/content/drive/MyDrive/PROJECT/chr_19_no_NaN.csv.gz'

df = pd.read_csv(file_path, sep='\t', dtype=str)
df.set_index('snpid', inplace=True)
df

,003_S_1057,003_S_0908,003_S_1122,136_S_0695,136_S_0873,130_S_0886,012_S_1133,003_S_1074,037_S_0501,027_S_0074,...,137_S_4672,019_S_4549,018_S_4696,126_S_4686,130_S_4660,023_S_4501,070_S_4692,005_S_4707,003_S_4350,141_S_4232
snpid,,,,,,,,,,,,,,,,,,,,,
rs13235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.5,0,0.5
rs4919862,1,0.5,0,0.5,0,0,0,0,0,0.5,...,0,0,0,0,0.5,0,0,0.5,0,0.5
rs8259,0,0,0,0.5,0,0,0,0,0,0,...,0,0,0,0,0.5,0,0,0,0,0
rs11473,0.5,0,0,0.5,0,0,0,0,0,0.5,...,1,0,0,0,0,0,0,0,0,0
rs6757,0,0.5,0,0,0.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs1654410,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
rs1671153,0,0,0.5,0,0,0,0,0.5,0,0,...,0,0.5,0,0,0,0,0,0,0,1
rs1654419,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
# Percorso del file GZIP
file_path = '/content/drive/MyDrive/PROJECT/lookup_snpid_gene.txt.gz'

gene_snp = pd.read_csv(file_path, sep='\t', dtype=str)
gene_snp[gene_snp['gene_name'] == 'None'] = np.nan
gene_snp = gene_snp.dropna()
gene_snp = gene_snp.set_index('snpid')
usefull_snps = df.index.tolist()
gene_snp = gene_snp.loc[usefull_snps]
gene_snp

# genes_to_be_removed = ['THRB', 'LIN28B', 'PGPEP1', 'GLTSCR1', 'IFITM2', 'NUDC', 'SDHA', 'STX8', 'GFM2', 'C19orf80', 'KIR3DL1', 'RIT1', 'LPP']




,gene_name
snpid,
rs13235,dnaK
rs4919862,BSG
rs8259,BSG
rs11473,BSG
rs6757,BSG
...,...
rs1654410,GP6
rs1671153,GP6
rs1654419,GP6


In [ ]:
#from this it was clear that the gene were also added to the network if not present there are 13 genes that are not in the snpid-gene df but there are in the nodes so the string database bult a network also adding those genes in particular the genes are 
#THRB 
#LIN28B
#PGPEP1
#GLTSCR1
#IFITM2
#NUDC
#SDHA
#STX8
#GFM2
#C19orf80
#KIR3DL1
#RIT1
#LPP
#while the genes that are not included inside the string database (belongs to gene_snp but are not in the nodes) are
#dnaK
#HCN2
#yciH
#CDKN2B-AS1
#MIR526B
#LOC128772423
#LOC128772424
#LOC127890069
#frc
#PLIN4
#TINCR
#TINCR
#LOC117307477
#LOC117307478
#thrB
#TGFBR3L
#rpmH
#metG
#potC
#OR7G3
#OR7D4
#ZNF699
#rrf
#pdxJ
#mscS
#LOC127890416
#LOC126862854
#ANGPTL8
#traL
#MIR23A
#MIR146A
#ADGRL1
#UCA1
#narI
#MIR3188
#GDF1
#ftsX
#trmB
#tadA
#thiI
#pgi
#ZNF208
#ZNF208
#ZNF208
#ZNF208
#thrS
#carB
#ptsP
#lpxC
#remA
#xapA
#sctF
#argG
#panM
#CYRIA
#RHPN2
#ompR
#pbuO
#miaB
#fnr
#crp
#crp
#LOC127891453
#ppnN
#yihA
#pspB
#rlmH
#ramB
#LOC110673974
#LOC110673974
#LOC110599575
#PCAT19
#ipdC
#mreB
#sdjA
#BICRA
#wzc
#LOC127891910
#lpp
#PCDH15
#PCDH15
#KLK1
#KLK1
#KLK1
#KLK1
#KLK5
#KLK12
#sdhA
#nudC
#rnk
#ccoS
#eboE
#MIR125A
#MIR21
#ZNF480
#lipY
#tauA
#MIR520F
#MIR371A
#HULC
#trmY
#motA
#ribB
#yejK
#LILRA3
#BW4

In [33]:
indici_comuni = gene_snp.loc[gene_snp['gene_name'].isin(list(lista_dizionari[0]['Graph'].nodes))].index
dataset = df.loc[indici_comuni]
dataset = dataset.T
dataset



snpid,rs4919862,rs8259,rs11473,rs6757,rs11085226,rs1683563,rs10407968,rs2240158,rs3764650,rs4147929,...,rs383369,rs1061680,rs41308748,rs3745902,rs147585490,rs1654410,rs1671153,rs1654419,rs1613662,rs12611091
003_S_1057,1,0,0.5,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
003_S_0908,0.5,0,0,0.5,0,0,0,1,0,0.5,...,0.5,0,0,0,1,1,0,1,0,0
003_S_1122,0,0,0,0,0,0,0,0,0,0.5,...,0,0,0,0,0.5,1,0.5,0,0,0
136_S_0695,0.5,0.5,0.5,0,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
136_S_0873,0,0,0,0.5,0,0.5,0,0,0,0,...,0,0,0,0.5,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
023_S_4501,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
070_S_4692,0,0,0,0,0,0,0,0,0.5,0,...,0,0,0,0,0,0,0,0,0,0
005_S_4707,0.5,0,0,0,0,0,0,1,0,0.5,...,0,0,0,0,0,0,0,0,0,0
003_S_4350,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [52]:
file_adnimerge = '/content/drive/MyDrive/PROJECT/output_adnimerge.csv'
adnimerge = pd.read_csv(file_adnimerge)
adnimerge = adnimerge.set_index('Unnamed: 0')
adnimerge

#mean_AGE = adnimerge['AGE'].mean()
# mean_AGE

,AGE,DX,PTGENDER,PTEDUCAT,PTETHCAT,APOE4,AV45,PIB,ABETA,TAU,...,PTAU+,APOE4+,AV45_scoring,PIB_scoring,TAU_slope,TAU_rvalue,PTAU_slope,PTAU_rvalue,FDG_slope,FDG_rvalue
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
003_S_5154,72.4,CN,Female,18,Not Hisp/Latino,1.0,1.0746,NaN,1159.0,190.6,...,0.0,1.0,0.811453,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_5165,78.8,Dementia,Male,12,Not Hisp/Latino,0.0,1.5762,NaN,886.2,627.9,...,1.0,0.0,1.699907,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_5187,61.4,Dementia,Female,16,Hisp/Latino,1.0,1.3999,NaN,725.1,379.0,...,1.0,1.0,1.121339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_5209,76.1,CN,Male,15,Not Hisp/Latino,0.0,1.0392,NaN,1139.0,218.1,...,0.0,0.0,0.846026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_6014,67.1,CN,Male,18,Not Hisp/Latino,0.0,1.1047,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941_S_7041,73.3,MCI,Female,20,Not Hisp/Latino,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941_S_7046,71.2,CN,Female,16,Not Hisp/Latino,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941_S_7051,63.7,CN,Male,12,Not Hisp/Latino,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Fem primer amb PET
adnimerge = label_ADNI(adnimerge,'PET')
adnimerge = adnimerge.T

merged_df = pd.concat([df, adnimerge], join='inner')
merged_df = merged_df.T
merged_df




,rs13235,rs4919862,rs8259,rs11473,rs6757,rs3752158,rs11085226,rs1683563,rs13090,rs10407968,...,rs143159382,rs35974949,rs3745902,rs147585490,rs1654410,rs1671153,rs1654419,rs1613662,rs12611091,y
136_S_0873,0,0,0,0,0.5,0.5,0,0.5,0,0,...,0,0,0.5,0,0,0,0,0,1,1.0
130_S_0886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.5,1.0
027_S_0074,0,0.5,0,0.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
051_S_1072,0,0,0,0,0.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
016_S_1117,0.5,1,0,0,0.5,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130_S_4660,0,0.5,0.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
023_S_4501,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.0
070_S_4692,0,0,0,0,0,0,0,0,0,0,...,0.5,0,0,0,0,0,0,0,0,1.0
005_S_4707,0.5,0.5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [54]:
file_adnimerge = '/content/drive/MyDrive/PROJECT/output_adnimerge.csv'
adnimerge = pd.read_csv(file_adnimerge)
adnimerge = adnimerge.set_index('Unnamed: 0')
adnimerge

,AGE,DX,PTGENDER,PTEDUCAT,PTETHCAT,APOE4,AV45,PIB,ABETA,TAU,...,PTAU+,APOE4+,AV45_scoring,PIB_scoring,TAU_slope,TAU_rvalue,PTAU_slope,PTAU_rvalue,FDG_slope,FDG_rvalue
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
003_S_5154,72.4,CN,Female,18,Not Hisp/Latino,1.0,1.0746,NaN,1159.0,190.6,...,0.0,1.0,0.811453,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_5165,78.8,Dementia,Male,12,Not Hisp/Latino,0.0,1.5762,NaN,886.2,627.9,...,1.0,0.0,1.699907,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_5187,61.4,Dementia,Female,16,Hisp/Latino,1.0,1.3999,NaN,725.1,379.0,...,1.0,1.0,1.121339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_5209,76.1,CN,Male,15,Not Hisp/Latino,0.0,1.0392,NaN,1139.0,218.1,...,0.0,0.0,0.846026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
003_S_6014,67.1,CN,Male,18,Not Hisp/Latino,0.0,1.1047,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941_S_7041,73.3,MCI,Female,20,Not Hisp/Latino,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941_S_7046,71.2,CN,Female,16,Not Hisp/Latino,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941_S_7051,63.7,CN,Male,12,Not Hisp/Latino,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
for i in merged_df.index:
    if i in adnimerge.index:
        merged_df.loc[i, 'PTGENDER'] = adnimerge.loc[i, 'PTGENDER']
        merged_df.loc[i, 'AGE'] = adnimerge.loc[i, 'AGE']

        
def gender_to_numeric(gender):
    if gender == 'Female':
        return 0
    elif gender == 'Male':
        return 1
    else:
        return None

merged_df['PTGENDER'] = merged_df['PTGENDER'].apply(gender_to_numeric)

# 54-60: Categoría 1
# 61-65: Categoría 2
# 66-70: Categoría 3
# 71-75: Categoría 4
# 76-80: Categoría 5
# 81-85: Categoría 6
# 86-91: Categoría 7

bins = [54, 60, 65, 70, 75, 80, 85, 91.5]
labels = [1, 2, 3, 4, 5, 6, 7]

merged_df['AGE'] = pd.cut(merged_df['AGE'], bins=bins, labels=labels)
merged_df['AGE'] = pd.to_numeric(merged_df['AGE'], errors='coerce')
merged_df

,rs13235,rs4919862,rs8259,rs11473,rs6757,rs3752158,rs11085226,rs1683563,rs13090,rs10407968,...,rs3745902,rs147585490,rs1654410,rs1671153,rs1654419,rs1613662,rs12611091,y,PTGENDER,AGE
136_S_0873,0,0,0,0,0.5,0.5,0,0.5,0,0,...,0.5,0,0,0,0,0,1,1.0,0,5
130_S_0886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.5,1.0,0,4
027_S_0074,0,0.5,0,0.5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,1,5
051_S_1072,0,0,0,0,0.5,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0,1
016_S_1117,0.5,1,0,0,0.5,0,0,0,0,0,...,0,0,0,0,1,0,1,1.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130_S_4660,0,0.5,0.5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,0,5
023_S_4501,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1.0,1,5
070_S_4692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1,6
005_S_4707,0.5,0.5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1,3


In [56]:
num_folds = 5
# Use KFold to generate the folds
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Create empty lists to store the folds
fold_dfs = []
i = 0
# Iterate over the folds
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(merged_df)):
    # Get the validation sets for this fold
    val_df = merged_df.iloc[val_idx].reset_index(drop=False)
    print(val_df.shape[0])
    val_df = val_df.rename(columns = {'index': 'snpid'})
    val_df = val_df.set_index('snpid')

# ignore_index=False  this is to keep the patients id

    # Store the folds in a list of DataFrames
    fold_dfs.append((val_df))


134
134
134
133
133


In [57]:
with open('/content/drive/MyDrive/PROJECT/dataset_folds_Age_Sex.pickle', 'wb') as handle:
    pickle.dump(fold_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
# checkejem que s'obri correctament: 
with open('/content/drive/MyDrive/PROJECT/dataset_folds_Age_Sex.pickle', 'rb') as handle:
    dataset = pickle.load(handle)
dataset[4]

,rs13235,rs4919862,rs8259,rs11473,rs6757,rs3752158,rs11085226,rs1683563,rs13090,rs10407968,...,rs3745902,rs147585490,rs1654410,rs1671153,rs1654419,rs1613662,rs12611091,y,PTGENDER,AGE
snpid,,,,,,,,,,,,,,,,,,,,,
130_S_0886,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.5,1.0,0,4
016_S_1117,0.5,1,0,0,0.5,0,0,0,0,0,...,0,0,0,0,1,0,1,1.0,0,3
127_S_1427,0,0,0,0,0,0,0,0,0,0,...,0.5,0,0,0,0,0,0.5,1.0,0,3
137_S_1414,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1.0,1,4
094_S_1417,0,0.5,0,0.5,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
073_S_4559,0,0,0,0,0,0,0.5,0,0,0,...,0,0,0,0,0,0.5,0,0.0,0,3
012_S_4643,0,0,0,0,0,0,1,0,0,0,...,0,0.5,0,0,0,0,0,0.0,0,3
013_S_4595,0.5,0,0.5,0,0,0,0,0.5,0,0,...,0.5,0.5,0.5,0,0,0,0,1.0,1,5
